In this file, I would like to study the Linear Regression problem. But I will study it, first without any packages (just numpy and pandas), create a function to compute the Regression and so on. After that, I will discuss how implemented and what are the differences to use Sci-kit learn, Tensorflow or Pytorch.

---
# **Regression problem**

Let start with the definition of the Regression problem. 

The regression problem is